In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 13 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,702 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [18]:
import numpy as np
import cv2
from imutils.object_detection import non_max_suppression
import pytesseract
import argparse
import time
import sys
from PIL import Image
from scipy.ndimage import interpolation as inter
from matplotlib import pyplot as plt
import tempfile

In [37]:
from google.colab import files
uploaded = files.upload()

Saving 10.png to 10.png


In [64]:
image_path_in_colab="/content/8.png"

In [65]:
args = {"image":"../input/text-detection/example-images/Example-images/ex24.jpg", "east":"../content/east_text_detection.pb", "min_confidence":0.5, "width":320, "height":320,"padding":0.0}

In [66]:
IMAGE_SIZE = 1800


def set_image_dpi(file_path):

    im = Image.open(file_path)
    length_x, width_y = im.size
    factor = max(1, int(IMAGE_SIZE / length_x))
    size = factor * length_x, factor * width_y
    # size = (1800, 1800)
    im_resized = im.resize(size, Image.ANTIALIAS)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
    temp_filename = temp_file.name
    im_resized.save(temp_filename, dpi=(300, 300))
    return temp_filename
  

#img= cv2.imread(img)
#im = Image.open(img)

#show image
#im.show()

In [67]:
#noise removal and smoothening
BINARY_THREHOLD = 180

def image_smoothening(img):
    ret1, th1 = cv2.threshold(img, BINARY_THREHOLD, 255, cv2.THRESH_BINARY)
    ret2, th2 = cv2.threshold(th1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    blur = cv2.GaussianBlur(th2, (1, 1), 0)
    ret3, th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return th3

def remove_noise_and_smooth(file_name):
    img = cv2.imread(file_name, 0)
    filtered = cv2.adaptiveThreshold(img.astype(np.uint8), 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, 41)
    kernel = np.ones((1, 1), np.uint8)
    opening = cv2.morphologyEx(filtered, cv2.MORPH_OPEN, kernel)
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)
    img = image_smoothening(img)
    or_image = cv2.bitwise_or(img, closing)
    return or_image

In [68]:
#skew correction

def correct_skew(image, delta=1, limit=5):
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        histogram = np.sum(data, axis=1)
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2)
        return histogram, score

    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] 

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for angle in angles:
        histogram, score = determine_score(thresh, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, \
              borderMode=cv2.BORDER_REPLICATE)

    return best_angle, rotated

In [69]:
#contouring

In [70]:
img_dpi = set_image_dpi(image_path_in_colab)
img = cv2.imread(img_dpi)
args["image"]=img_dpi
angle, rotated = correct_skew(img)
orig = rotated.copy()
(origH, origW) = rotated.shape[:2]
# set the new width and height and then determine the ratio in change
# for both the width and height
(newW, newH) = (args["width"], args["height"])
rW = origW / float(newW)
rH = origH / float(newH)
# resize the image and grab the new image dimensions
image = cv2.resize(img, (newW, newH))
(H, W) = image.shape[:2]
#img_dpi = Image.fromarray(resizelist[val])
#img_smooth = remove_noise_and_smooth(img_dpi)

In [71]:

extractedInformation = pytesseract.image_to_string(Image.open(img_dpi))
print(extractedInformation)

Foreword

The need to get the best possible value from spending public money will
always remain a constant for those entrusted with spending decisions. The
need to reduce overall spending resulting from the financial crisis of 2008 has
sharpened this requirement. The continuing downward pressure on the
availability of public sector finance together with the ever growing upward
pressures of demand for public services will continue to further increase the
need to make better use of the resources available, the challenge has never
been greater.

In this context it is vital that capital spending decisions are taken on the basis
of highly competent professionally developed spending proposals. This
Treasury guidance which has been refined and tested over many years
provides a clear framework for thinking about spending proposals and a
structured process for appraising, developing and planning to deliver best
public value. All of which is captured through a well prepared business case
which s

In [72]:
print(pytesseract.image_to_boxes(Image.open(image_path_in_colab)))

F 26 538 30 548 0
o 30 538 38 548 0
r 38 538 46 548 0
e 46 538 53 548 0
w 53 538 61 548 0
o 61 538 66 548 0
r 70 538 77 549 0
d 77 538 88 549 0
‘ 527 0 527 0 0
T 26 507 29 517 0
h 29 507 38 517 0
e 38 507 48 517 0
n 56 507 59 517 0
e 59 507 67 517 0
e 67 507 74 517 0
d 74 507 86 517 0
t 93 507 94 516 0
o 94 507 103 516 0
g 112 504 116 516 0
e 116 504 121 516 0
t 121 504 129 516 0
t 137 507 139 517 0
h 139 507 145 517 0
e 145 507 153 517 0
b 162 507 166 517 0
e 166 507 174 517 0
s 174 507 177 517 0
t 179 506 188 516 0
p 195 504 198 515 0
o 198 504 203 515 0
s 203 504 213 515 0
s 213 504 218 515 0
i 218 504 223 515 0
b 223 504 223 515 0
l 230 507 236 517 0
e 236 507 244 517 0
v 251 507 254 514 0
a 254 507 258 514 0
l 259 506 268 517 0
u 268 506 268 517 0
e 273 507 283 515 0
f 290 507 293 517 0
r 293 507 297 517 0
o 297 507 306 517 0
m 306 507 316 517 0
s 324 504 328 517 0
p 328 504 335 517 0
e 335 504 342 517 0
n 342 504 351 517 0
d 351 504 355 517 0
i 355 504 360 517 0
n 360 504 360 517

In [74]:
file1 = open("myfile2.txt","w") 
file1.writelines(extractedInformation) 